In [2]:
from pydataset import data
import pandas as pd
import numpy as np

import datetime

In [ ]:
# 1. Load the mpg dataset. Read the documentation for it, and use the data to answer 
# these questions:

data('mpg', show_doc=True) # view the documentation for the dataset
# with show_doc=True won't assign data to variable

mpg = data('mpg') # load the dataset and store it in a variable



In [ ]:
# On average, which manufacturer has the best miles per gallon?

# add column of average mpg
mpg['avg_mpg'] = (mpg.cty + mpg.hwy) / 2

# average mpg per manufacturer best on top
mpg.groupby('manufacturer').avg_mpg.mean().sort_values(ascending=False)


In [ ]:
# How many different manufacturers are there?

# use .unique() to get list of unique values, len() will give total count
# https://cmdlinetips.com/2018/01/how-to-get-unique-values-from-a-column-in-pandas-data-frame/
len(mpg.manufacturer.unique())

In [ ]:
# How many different models are there?
# use .unique() to get list of unique values, len() will give total count
len(mpg.model.unique())

In [ ]:
# Do automatic or manual cars have better miles per gallon?
mpg.head()
bools = mpg.trans.str.contains('man')
print("Cars with a manual transmission = True, Automatic Transmission = False")
print("Cars with a manual transmission have a higher average mpg")
mpg.groupby(bools).avg_mpg.mean().sort_values(ascending=False)

In [ ]:
# 2. Joining and Merging
# Copy the users and roles dataframes from the examples above. 

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})


# What do you think a right join would look like? 

# a right join would show all roles (even those without users), but not all users
pd.merge(users, roles, left_on='role_id', right_on='id', how='right')


In [ ]:
#  An outer join?

# an outer join should show all users and all roles inserting null values where needed
pd.merge(users, roles, left_on='role_id', right_on='id', how='outer')

In [ ]:
# What happens if you drop the foreign keys from the dataframes and try to merge them?

# it puts the roles as additional names below the original users
pd.merge(users, roles, how='outer')

In [3]:
# 3. Getting data from SQL databases
# 3.a. Create a function named get_db_url. It should accept a username, hostname, 
# password, and database name and return a url formatted like in the examples 
# in this lesson.

# from env import host, user, password
# url = f'mysql+pymysql://{user}:{password}@{host}/employees'
# pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)


# for getting from Codeup SQL databases
def get_db_url(database):
    from env import host, user, password
    url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return url

# note: this would require typing user and password into jupyter notebook
# which would defeat the purpose of annonimity!
# def general_get_db_url(user, password, host, database):
#     url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
#     return url


In [ ]:
# Ryan's challenge = show all available databases
from env import host, user, password
url = f'mysql+pymysql://{user}:{password}@{host}'
query = '''show databases'''
pd.read_sql(query, url)

In [ ]:
# 3.b. Use your function to obtain a connection to the employees database.
# (and run a query)

query = """select * from orders limit 100"""
pd.read_sql(query, get_db_url('chipotle'))


In [ ]:
# 3. Once you have successfully run a query:
# 3.c.i. Intentionally make a typo in the database url. What kind of error message do you see?

# misspell database name
#query = """select * from orders limit 100"""
#pd.read_sql(query, get_db_url('chipotel'))

# spelling error in url, supposed to be pymysql
url = f'mysql+pysql://{user}:{password}@{host}'
query = '''show databases'''
pd.read_sql(query, url)


In [ ]:
# 3.c.ii. Intentionally make an error in your SQL query. What does the error message look like?

# misspell query table name
query = """select * from oders limit 100"""
pd.read_sql(query, get_db_url('chipotle'))


In [3]:
# 3.d. Read the employees and titles tables into two separate dataframes

# write SQL queries for tables
query_employees = '''select * from employees'''
query_titles = '''select * from titles'''
# url function
url = get_db_url('employees')
# assign tables to df variables
employeesdf = pd.read_sql(query_employees, url)
titlesdf = pd.read_sql(query_titles, url)


In [ ]:
# 3.e. Visualize the number of employees with each title.

# 3.e.i. current data
# get a sense of what titlesdf looks like
titlesdf.head()
# set today's date variable and create bools to filter to_date greater than today
today = pd.to_datetime('today')
bools = titlesdf.to_date > today 
# use the groupby to count the number of employees with that title currently
titlesdf[bools].groupby('title').count()

In [ ]:
# 3.e.ii. Visualize the number of employees with each title -- for all titles historical
# this will count the historical number of employees with that title
titlesdf.groupby('title').count()

In [4]:
# 3.f. Join the employees and titles dataframes together.

# get a sense of the columns in the df to join
print(employeesdf.head())
print(titlesdf.head())

# merge dfs on emp_no
emp_with_titles = pd.merge(employeesdf, titlesdf, left_on='emp_no', right_on='emp_no', how='outer')
# show combined head
emp_with_titles.head()

   emp_no  birth_date first_name last_name gender   hire_date
0   10001  1953-09-02     Georgi   Facello      M  1986-06-26
1   10002  1964-06-02    Bezalel    Simmel      F  1985-11-21
2   10003  1959-12-03      Parto   Bamford      M  1986-08-28
3   10004  1954-05-01  Chirstian   Koblick      M  1986-12-01
4   10005  1955-01-21    Kyoichi  Maliniak      M  1989-09-12
   emp_no            title   from_date     to_date
0   10001  Senior Engineer  1986-06-26  9999-01-01
1   10002            Staff  1996-08-03  9999-01-01
2   10003  Senior Engineer  1995-12-03  9999-01-01
3   10004         Engineer  1986-12-01  1995-12-01
4   10004  Senior Engineer  1995-12-01  9999-01-01


,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


In [5]:
# 3.g. Visualize how frequently employees change titles.
# find type of data in to_date and from_date columns, use index 1 to get data type for specific date
type(emp_with_titles.to_date[1])
# exobrain http://chris35wills.github.io/time_elapsed_pandas/
# columns are objects, pandas timedelta works on objects
# per exobrain https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timedelta.html
# bools mask to find just those with start and end date for row
today = pd.to_datetime('today')
bools = titlesdf.to_date < today 
# create column to show bools mask applied
emp_with_titles['calc_date'] = emp_with_titles.to_date[bools]
# subtract calculation date and from(start) date and convert to years
# exobrain http://www.datasciencemadesimple.com/difference-two-dates-days-weeks-months-years-pandas-python-2/
#emp_with_titles['time_in_position'] = emp_with_titles['to_date'] - emp_with_titles['from_date']
#emp_with_titles['time_in_position']=df['diff_years']/np.timedelta64(1,'Y')
emp_with_titles['time_in_title'] = (emp_with_titles.calc_date - emp_with_titles.from_date)/np.timedelta64(1,'Y')
# create new df, groupby title and get aggregate mean
title_freqdf = emp_with_titles.groupby('title').mean()
# remove emp_no from display of new df
title_freqdf = title_freqdf.drop(columns='emp_no')
# round df to 2 digits
title_freqdf = title_freqdf.round(2)
title_freqdf

,time_in_title
title,
Assistant Engineer,6.04
Engineer,6.00
Manager,4.72
Senior Engineer,4.10
Senior Staff,4.05
Staff,6.07
Technique Leader,4.83


In [ ]:
# 3.h. For each title, find the hire date of the employee that was hired most recently 
# with that title.

# check dataframe
emp_with_titles.head()
# drop added columns, ran the first time then commented out
# emp_with_titles = emp_with_titles.drop(columns=['calc_date', 'time_in_title'])
# check dataframe
emp_with_titles.head()
# show most recent date hired by title
emp_with_titles.groupby('title').hire_date.max()

In [19]:
# 3.i. Write the code necessary to create a cross tabulation of the number of titles
# by department. (Hint: this will involve a combination of SQL and python/pandas code)
# write SQL queries for tables

# get department and current dept_emp from SQL and join on dept_no
query_dept = '''select * from departments'''
query_dept_emp = '''select * from dept_emp where to_date > current_date'''
deptdf = pd.read_sql(query_dept, url)
dept_empdf = pd.read_sql(query_dept_emp, url)
deptdf = pd.merge(deptdf, dept_empdf, left_on='dept_no', right_on='dept_no', how='outer')

# continued below

In [21]:
# 3.i. continued

# get current titles
query_titles = '''select * from titles where to_date > current_date'''
titledf = pd.read_sql(query_titles, url)
print(deptdf.head())
print(titledf.head())

# # join on emp_no
dept_titledf = pd.merge(deptdf, titledf, left_on='emp_no', right_on='emp_no') 
# # check new df shape
dept_titledf.head()

# continued below

  dept_no         dept_name  emp_no   from_date     to_date
0    d009  Customer Service   10038  1989-09-20  9999-01-01
1    d009  Customer Service   10049  1992-05-04  9999-01-01
2    d009  Customer Service   10060  1992-11-11  9999-01-01
3    d009  Customer Service   10088  1992-03-21  9999-01-01
4    d009  Customer Service   10112  1998-05-01  9999-01-01
   emp_no            title   from_date     to_date
0   10001  Senior Engineer  1986-06-26  9999-01-01
1   10002            Staff  1996-08-03  9999-01-01
2   10003  Senior Engineer  1995-12-03  9999-01-01
3   10004  Senior Engineer  1995-12-01  9999-01-01
4   10005     Senior Staff  1996-09-12  9999-01-01


,dept_no,dept_name,emp_no,from_date_x,to_date_x,title,from_date_y,to_date_y
0,d009,Customer Service,10038,1989-09-20,9999-01-01,Senior Staff,1996-09-20,9999-01-01
1,d009,Customer Service,10049,1992-05-04,9999-01-01,Senior Staff,2000-05-04,9999-01-01
2,d009,Customer Service,10060,1992-11-11,9999-01-01,Senior Staff,1996-05-28,9999-01-01
3,d009,Customer Service,10088,1992-03-21,9999-01-01,Senior Staff,1993-09-02,9999-01-01
4,d009,Customer Service,10112,1998-05-01,9999-01-01,Staff,1998-05-01,9999-01-01


In [22]:
# 3.i. continued

# crosstab dept_name and title
pd.crosstab(dept_titledf.dept_name, dept_titledf.title)


title,Assistant Engineer,Engineer,Manager,Senior Engineer,Senior Staff,Staff,Technique Leader
dept_name,,,,,,,
Customer Service,68,627,1,1790,11268,3574,241
Development,1652,14040,1,38816,1085,315,5477
Finance,0,0,1,0,9545,2891,0
Human Resources,0,0,1,0,9824,3073,0
Marketing,0,0,1,0,11290,3551,0
Production,1402,12081,1,33625,1123,349,4723
Quality Management,389,3405,1,9458,0,0,1293
Research,77,830,1,2250,9092,2870,321
Sales,0,0,1,0,28797,8903,0


In [4]:
# 4. Use your get_db_url function to help you explore the data from the chipotle database. 
# Use the data to answer the following questions:


query_chip = '''select * from orders'''
# url function
url = get_db_url('chipotle')
# assign table to df
chipdf = pd.read_sql(query_chip, url)

chipdf.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


In [5]:
# 4.a. What is the total price for each order?

# dtype item_price
type(chipdf.item_price[1])

# clean and change item_price to float
# exobrain https://www.kite.com/python/answers/how-to-convert-a-pandas-dataframe-column-of-strings-to-floats-in-python
chipdf.item_price = chipdf.item_price.str.replace('$', '')
# example: df["A"] = pd.to_numeric(df["A"], downcast="float")
chipdf['item_price'] = pd.to_numeric(chipdf['item_price'], downcast='float')

# review of data and basic knowledge of Chipotle pricing indicates that item_price is already inclusive of
# quantity * base item price

# continued below

In [47]:
# 4.a. continued

# group by order_id and sum -- quantity is # of items, item_price is order total $
chipdf.groupby('order_id').sum()

,id,quantity,item_price
order_id,,,
1,10,4,11.56
2,5,2,16.98
3,13,2,12.67
4,17,2,21.00
5,21,2,13.70
...,...,...,...
1830,9223,2,23.00
1831,13842,3,12.90
1832,9233,2,13.20


In [6]:
# 4.b. What are the most popular 3 items?

# use group by and sum item quantity the sort descending to largest 3 at top of list
chipdf.groupby('item_name')[['quantity']].sum().sort_values(by='quantity', ascending=False).head()


,quantity
item_name,
Chicken Bowl,761
Chicken Burrito,591
Chips and Guacamole,506
Steak Burrito,386
Canned Soft Drink,351


In [7]:
# 4.c. Which item has produced the most revenue?

# get sense of data
chipdf.head()
# use group by like above, but on item price instead of quantity
chipdf.groupby('item_name')[['item_price']].sum().sort_values(by='item_price', ascending=False).head()

,item_price
item_name,
Chicken Bowl,7342.729980
Chicken Burrito,5575.819824
Steak Burrito,3851.429932
Steak Bowl,2260.189941
Chips and Guacamole,2201.040039
